In [1]:
# iris dataset
from sklearn import datasets
import pandas as pd
from collections import namedtuple

In [2]:
Data = namedtuple('Data', ['features', 'target', 'target1D'])
def load_iris():
    dataset = datasets.load_iris()
    features = pd.DataFrame(dataset.data, columns=dataset.feature_names)
    target = pd.DataFrame(pd.get_dummies(dataset.target))
    target.columns = dataset.target_names
    assert features.shape[0] == target.shape[0]
    print('The Iris dataset was used in R.A. Fisher\'s classic 1936 paper *The Use of Multiple Measurements in Taxonomic Problems*.')
    print('')
    print('features.shape = {}'.format(features.shape))
    print('target.shape = {}'.format(target.shape))
    return Data(features, target, pd.DataFrame(dataset.target, columns=['class']))


load_iris()


The Iris dataset was used in R.A. Fisher's classic 1936 paper *The Use of Multiple Measurements in Taxonomic Problems*.

features.shape = (150, 4)
target.shape = (150, 3)


Data(features=     sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
0                  5.1               3.5                1.4               0.2
1                  4.9               3.0                1.4               0.2
2                  4.7               3.2                1.3               0.2
3                  4.6               3.1                1.5               0.2
4                  5.0               3.6                1.4               0.2
..                 ...               ...                ...               ...
145                6.7               3.0                5.2               2.3
146                6.3               2.5                5.0               1.9
147                6.5               3.0                5.2               2.0
148                6.2               3.4                5.4               2.3
149                5.9               3.0                5.1               1.8

[150 rows x 4 columns], target=     setosa  versi

In [20]:
from sklearn import datasets
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV


iris = datasets.load_iris() #Loading the dataset
iris.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])

In [23]:
X, y = iris.data, iris.target
# Splitting into train and test
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)

In [22]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


In [24]:
pipe = Pipeline([('scaler', StandardScaler()), ('clf', RandomForestClassifier())])

In [30]:
?GridSearchCV

Init signature:
GridSearchCV(
    estimator,
    param_grid,
    *,
    scoring=None,
    n_jobs=None,
    refit=True,
    cv=None,
    verbose=0,
    pre_dispatch='2*n_jobs',
    error_score=nan,
    return_train_score=False,
)
Docstring:     
Exhaustive search over specified parameter values for an estimator.

Important members are fit, predict.

GridSearchCV implements a "fit" and a "score" method.
It also implements "score_samples", "predict", "predict_proba",
"decision_function", "transform" and "inverse_transform" if they are
implemented in the estimator used.

The parameters of the estimator used to apply these methods are optimized
by cross-validated grid-search over a parameter grid.

Read more in the :ref:`User Guide <grid_search>`.

Parameters
----------
estimator : estimator object
    This is assumed to implement the scikit-learn estimator interface.
    Either estimator needs to provide a ``score`` function,
    or ``scoring`` must be passed.

param_grid : dict or list of

In [37]:
grid = GridSearchCV(pipe, param_grid={'clf__n_estimators': [10, 100, 1000], 'clf__criterion': ['gini', 'entropy'] }, cv=5)
res = grid.fit(X_train, y_train) 

In [36]:
grid.best_params_

{'clf__criterion': 'gini', 'clf__n_estimators': 1000}

In [38]:
pd.DataFrame(res.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf__criterion,param_clf__n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.027800,0.006431,0.002601,0.000800,gini,10,"{'clf__criterion': 'gini', 'clf__n_estimators'...",0.958333,0.958333,0.833333,1.0,0.916667,0.933333,0.056519,6
1,0.149258,0.033208,0.010379,0.001506,gini,100,"{'clf__criterion': 'gini', 'clf__n_estimators'...",0.958333,0.958333,0.833333,1.0,0.958333,0.941667,0.056519,4
2,1.317145,0.085635,0.100648,0.016360,gini,1000,"{'clf__criterion': 'gini', 'clf__n_estimators'...",0.958333,1.000000,0.875000,1.0,0.958333,0.958333,0.045644,1
3,0.012407,0.000794,0.001398,0.000493,entropy,10,"{'clf__criterion': 'entropy', 'clf__n_estimato...",0.958333,1.000000,0.833333,1.0,0.958333,0.950000,0.061237,2
4,0.183196,0.029427,0.014797,0.002040,entropy,100,"{'clf__criterion': 'entropy', 'clf__n_estimato...",0.958333,0.958333,0.833333,1.0,0.958333,0.941667,0.056519,4
5,1.480838,0.217572,0.098586,0.007631,entropy,1000,"{'clf__criterion': 'entropy', 'clf__n_estimato...",0.958333,0.958333,0.875000,1.0,0.958333,0.950000,0.040825,2


In [39]:
# pipe.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('clf', RandomForestClassifier())])

In [26]:
# pipe.score(X_test, y_test)

1.0

In [40]:
final_pipline = res.best_estimator_

In [41]:
from joblib import dump, load
dump(final_pipline, 'iris_pipeline.joblib')

['iris_pipeline.joblib']

In [42]:
iris_pipeline_2 = load('iris_pipeline.joblib')